<h1>Dates table</h1>

<h3>IMPORTS</h3>

In [ ]:
#IMPORTS
import pandas as pd
import sqlite3
from datetime import datetime, timedelta

<h3>METHODS</h3>

In [ ]:
def get_dates_until_current(start_date_str):
    """
    Generates a list of dates from a given start date up to the current date.

    Args:
        start_date_str (str): The start date in 'YYYY-MM-DD' format.

    Returns:
        list: A list of datetime.date objects representing the dates in the range.
    """
    start_date = datetime.strptime(start_date_str, '%Y-%m-%d').date()
    current_date = datetime.now().date()
    date_list = []

    # Iterate from the start_date until it exceeds the current_date
    while start_date <= current_date:
        date_list.append(start_date)
        start_date += timedelta(days=1) # Increment by one day

    return date_list



In [ ]:
def create_dates_df(dates_in_range):

    # Create DataFrame
    dates_df = pd.DataFrame(dates_in_range, columns=["Date"])

    #convert to pandas datetime format
    dates_df["Date"] = pd.to_datetime(dates_df["Date"])

    #extract granularity from dates
    dates_df["Month"] = dates_df["Date"].dt.month
    dates_df["Year"] = dates_df["Date"].dt.year
    dates_df["Quarter"] = dates_df["Date"].dt.quarter

    return dates_df

<h3>CREATE DATAFRAME</h3>

In [ ]:
#Get all dates from start of record to current date
start_date_input = '2015-01-01' #YYYY-MM-DD format

#check if earliest date
cursor = db_conn.cursor()


sql_earliest_date = "SELECT Min(Date) AS OldestDate from weather_data;"


try:
    # Execute the DELETE statement
    cursor.execute(sql_create_unique_table)
    cursor.execute(sql_drop_old_table)
    cursor.execute(sql_create_new_table)
    cursor.execute(sql_drop_temp_table)


    # Commit the changes to the database
    db_conn.commit()

    print(f"Distinct '{db_table}' table has been created.")

except sqlite3.Error as e:
    print(f"Error creating distinct table: {e}")

finally:
    # Close the cursor
    cursor.close()


dates_in_range = get_dates_until_current(start_date_input)

dates_df = create_dates_df(dates_in_range)

Distinct 'weather_data' table has been created.


<h3>Establish file paths & connections</h3>

In [ ]:
# File paths
data_folder_path = "./../Data/"
db_file_path = "./../WeatherStationDB.db"

#connection to database
db_conn = sqlite3.connect(db_file_path)
db_table = 'Dates'

<h3>RESET TABLE ROWS</h3>

In [ ]:
cursor = db_conn.cursor()


sql_create_unique_table = "CREATE TABLE dates_temp AS SELECT DISTINCT * FROM Dates"
sql_drop_old_table = "DROP TABLE Dates"
sql_create_new_table = "CREATE TABLE Dates AS SELECT DISTINCT * FROM dates_temp"
sql_drop_temp_table = "DROP TABLE dates_temp"

try:
    # Execute the DELETE statement
    cursor.execute(sql_create_unique_table)
    cursor.execute(sql_drop_old_table)
    cursor.execute(sql_create_new_table)
    cursor.execute(sql_drop_temp_table)


    # Commit the changes to the database
    db_conn.commit()

    print(f"Distinct '{db_table}' table has been created.")

except sqlite3.Error as e:
    print(f"Error creating distinct table: {e}")

finally:
    # Close the cursor
    cursor.close()

Distinct 'Dates' table has been created.


<h3>INSERT ALL Data into weather_data table</h3>

In [ ]:
# 'if_exists' can be 'fail', 'replace', or 'append'
# 'index=False' prevents writing the DataFrame index as a column in the database
dates_df.to_sql(db_table, db_conn, if_exists='append', index=False)

3892

In [ ]:
# cursor = db_conn.cursor()


# sql_create_unique_table = "CREATE TABLE stations_temp AS SELECT DISTINCT * FROM stations"
# sql_drop_old_table = "DROP TABLE stations"
# sql_create_new_table = "CREATE TABLE stations AS SELECT DISTINCT * FROM stations_temp"
# sql_drop_temp_table = "DROP TABLE stations_temp"

# try:
#     # Execute the DELETE statement
#     cursor.execute(sql_create_unique_table)
#     cursor.execute(sql_drop_old_table)
#     cursor.execute(sql_create_new_table)
#     cursor.execute(sql_drop_temp_table)


#     # Commit the changes to the database
#     db_conn.commit()

#     print(f"Distinct '{db_table}' table has been created.")

# except sqlite3.Error as e:
#     print(f"Error creating distinct table: {e}")

# finally:
#     # Close the cursor
#     cursor.close()